In [1]:
# --coding:utf-8--
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
# %matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from scipy import sparse
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import scipy as sp

def logloss(act, pred):
  epsilon = 1e-15
  pred = sp.maximum(epsilon, pred)
  pred = sp.minimum(1-epsilon, pred)
  ll = sum(act*sp.log(pred) + sp.subtract(1,act)*sp.log(sp.subtract(1,pred)))
  ll = ll * -1.0/len(act)
  return ll


path = '~/codedata/tpai/pre/'
trainfile = 'train.csv'
# testfile = 'test.csv'
train = pd.read_csv(path + trainfile)

In [6]:
trainNum = 100000
train = train.sample(n=trainNum, replace = True)

(100000, 8)

In [13]:
train.drop('clickTime', axis=1,inplace=True)
train.head()

,label,creativeID,connectionType,telecomsOperator
277574,0,4565,1,1
847118,0,6220,1,1
2704299,0,2235,2,1
3136110,0,6241,2,1
1761118,0,1128,1,1


In [14]:
connection_dummy = pd.get_dummies(train['connectionType'])
connection_dummy.head()

,0,1,2,3,4
277574,0,1,0,0,0
847118,0,1,0,0,0
2704299,0,0,1,0,0
3136110,0,0,1,0,0
1761118,0,1,0,0,0


In [15]:
train = train.join(connection_dummy)
train.head()

,label,creativeID,connectionType,telecomsOperator,0,1,2,3,4
7,0,1456,1,3,0,1,0,0,0
158,0,4793,2,1,0,0,1,0,0
195,0,3293,1,2,0,1,0,0,0
235,0,2537,1,2,0,1,0,0,0
238,0,5186,1,1,0,1,0,0,0


In [16]:
train.drop('creativeID',axis=1,inplace=True)

In [18]:
train.drop('connectionType',axis=1,inplace=True)
train.head()

,label,telecomsOperator,0,1,2,3,4
7,0,3,0,1,0,0,0
158,0,1,0,0,1,0,0
195,0,2,0,1,0,0,0
235,0,2,0,1,0,0,0
238,0,1,0,1,0,0,0


In [22]:
telecoms_dummy = pd.get_dummies(train['telecomsOperator'])
telecoms_dummy.columns = ['t0','t1','t2','t3']
# telecoms_dummy.head()
train = train.join(telecoms_dummy)
train.head()

,label,telecomsOperator,0,1,2,3,4,t0,t1,t2,t3
7,0,3,0,1,0,0,0,0,0,0,1
158,0,1,0,0,1,0,0,0,1,0,0
195,0,2,0,1,0,0,0,0,0,1,0
235,0,2,0,1,0,0,0,0,0,1,0
238,0,1,0,1,0,0,0,0,1,0,0


In [23]:
train.drop('telecomsOperator', axis=1, inplace=True)
train.head()

,label,0,1,2,3,4,t0,t1,t2,t3
7,0,0,1,0,0,0,0,0,0,1
158,0,0,0,1,0,0,0,1,0,0
195,0,0,1,0,0,0,0,0,1,0
235,0,0,1,0,0,0,0,0,1,0
238,0,0,1,0,0,0,0,1,0,0


In [29]:
# train.drop('label', axis=1)
X_train,X_test,Y_train,Y_test = train_test_split(train.drop('label',axis=1), \
                            train['label'].values, train_size=0.8)
print X_train.shape
print Y_train.shape
print X_test.shape
print Y_test.shape

(95025, 9)
(95025,)
(23757, 9)
(23757,)


In [30]:
# Random Forests  0.236746484137
random_forest = RandomForestRegressor(n_estimators=400)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
# random_forest.score(X_train, Y_train)
print logloss(Y_test, Y_pred)


0.116582099373
